# Exercise: Breweries Data from BreweryDB API

Using the Brewery DB API from the "Get Data from API - with key" lesson, extract the following data for the features of a brewery.

ID<br>
Name<br>
Year of Establishment<br>
Status<br>
Is it in business?<br>
Is it mass owned?<br>
Is it organic?<br>
Website<br>
Description

*HINT*: The URL to access the brewery side of the API is https://sandbox-api.brewerydb.com/v2/breweries/?key=

Based on the columns we want in the final dataframe, these are the corresponding locations...

So far, my best idea on how to do this involves first, grabbing what I need for the beer data so that I have the beer ids to throw into a request to Beer Brewery. Then, request the data for the brewery and append the data to the brewery relevant lists. Once complete, zipping it all up.

Sounds good in theory, let's see how it does in practice!

Here is a list of how things are appearing in the json structures to navigate to what is needed for each column.

ID = beerdata['data'][0]['id']<br>
Name = beerdata['data'][0]['nameDisplay']<br>
Year of Establishment = brewdata['data'][0]['established']<br>
Status = beerdata['data'][0]['statusDisplay']<br>
Is it in business? = brewdata['data'][0]['status']<br>
Is it mass owned? = brewdata['data'][0]['isMassOwned']<br>
Is it organic? = beerdata['data'][0]['isOrganic']<br>
Website = brewdata['data'][0]['website']<br>
Description = beerdata['data'][0]['description']

Not all beers *have* a description, so be careful!

In [3]:
import requests
import json
import pandas as pd

In [11]:
url = r"https://sandbox-api.brewerydb.com/v2/beers/?key="
# brewurl = r"https://sandbox-api.brewerydb.com/v2/breweries/?key="
# beerbrewurl = r"https://sandbox-api.brewerydb.com/v2/beer/" + :beerId + "/breweries"
# This will be needed later, as we populate the information we need from the main beers url

# We may not even really want to just pull all of the breweries, just the ones we want based on the beers pulled...
# brewurl = r"https://sandbox-api.brewerydb.com/v2/brewery/" + :breweryId

# At this point, you will need to go to the brewerydb.com website to create an account
# and acquire an api key. Then, create a file with just the api key inside named "brewDB_key.txt"
# kept in the same directory as this file.

with open('brewDB_key.txt', 'r') as file:
    for line in file:
        api_key = line
        
response = requests.get(url+api_key)
response

# brewresponse = requests.get(brewurl+api_key)
# brewresponse
# The API doc is out of date and I needed to pull brewery data to see what keys I can access,
# as the API does not show the key for 'isMassOwned'

<Response [200]>

In [12]:
beerdata = response.json()
type(beerdata)

#brewdata = brewresponse.json()

In [39]:
beers = beerdata['data']

beeridls = []
beernamels = []
beeryearls = []
beerstatusls = [] 
brewstatusls = [] 
brewownls = [] 
organicls = [] 
brewsitels = [] 
beerdescls = [] 

for beer in beers:
        
    #get the value of each key and if the key doesn't exist, set a variable to be None
    try: bid = beer['id']
    except: bid = None
            
    try: dname = beer['nameDisplay']
    except: dname = None
        
    try: beerstatus = beer['statusDisplay']
    except: beerstatus = None
            
    try: organic = beer['isOrganic']
    except: organic = None
        
    try: descr = beer['description']
    except: descr = None
        
    # We've gathered all we can from the beer structure of the final data we want, now it's time to pull from
    # Beer Breweries and ultimately Brewery based on the beer's ID
    
    beerbrewurl = r"https://sandbox-api.brewerydb.com/v2/beer/" + bid + "/breweries/?key="
    beerbrewresp = requests.get(beerbrewurl+api_key)
    beerbrewdata = beerbrewresp.json()
    
    breweryid = beerbrewdata['data'][0]['id']
    
    brewurl = r"https://sandbox-api.brewerydb.com/v2/brewery/" + breweryid + "/?key="
    brewresp = requests.get(brewurl+api_key)
    brewdata = brewresp.json()
    
    try: brewest = brewdata['data']['established']
    except: brewest = None
        
    try: brewstat = brewdata['data']['status']
    except: brewstat = None
            
    try: brewown = brewdata['data']['isMassOwned']
    except: brewown = None
        
    try: brewsite = brewdata['data']['website']
    except: brewsite = None
        
    beeridls.append(bid)
    beernamels.append(dname)
    beeryearls.append(brewest)
    beerstatusls.append(beerstatus)
    brewstatusls.append(brewstat)
    brewownls.append(brewown)
    organicls.append(organic)
    brewsitels.append(brewsite)
    beerdescls.append(descr)

In [40]:
colnames = ['ID','Name','Year of Establishment','Status','Is it in business?',
'Is it mass owned?','Is it organic?','Website','Description']

In [41]:
beerList = list(zip(beeridls, beernamels, beeryearls, beerstatusls, brewstatusls, brewownls, organicls, 
                    brewsitels, beerdescls))

In [42]:
df = pd.DataFrame(beerList, columns=colnames)

In [44]:
df.head(10)

,ID,Name,Year of Establishment,Status,Is it in business?,Is it mass owned?,Is it organic?,Website,Description
0,c4f2KE,'Murican Pilsner,1980,Verified,verified,N,N,http://www.sierranevada.com/,None
1,zTTWa2,11.5° PLATO,1980,Verified,verified,N,N,http://www.sierranevada.com/,The Plato scale is a measurement of the densit...
2,zfP2fK,12th Of Never,1993,Verified,verified,Y,N,http://www.lagunitas.com/,"Tropically Hoppy. Light, yet Full-Bodied. Brig..."
3,xwYSL2,15th Anniversary Ale,1997,Verified,verified,N,N,http://www.sweetwaterbrew.com/,For the ﬁrst ever SweetWater anniversary beer ...
4,UJGpVS,16 So Fine Red Wheat Wine,1997,Verified,verified,N,N,http://www.sweetwaterbrew.com/,For our super heady 16 year anniversary beer w...
5,vz5JZ1,1794 The Fergal Project,1994,Verified,verified,N,N,http://www.wachusettbrew.com/,None
6,LcpeBb,17th Saison,1997,Verified,verified,N,N,http://www.sweetwaterbrew.com/,This year’s anni ale has everything in it but ...
7,Yd8G8r,18th Anniversary Belgian Tripel,1997,Verified,verified,N,N,http://www.sweetwaterbrew.com/,This brilliant Belgian Tripel showcases the be...
8,iJ329Y,19 - Golden Belgian Style Ale,1997,Verified,verified,N,N,http://www.sweetwaterbrew.com/,This Golden Belgian Style Ale matured for over...
9,O78ldO,1904 American Red Lager,1852,Verified,verified,Y,N,http://www.anheuser-busch.com/,1904 American Red Lager is a special beer that...
